In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
# Inspecting the head of the dataframe to see what what variables we have

In [ ]:
df.head()

In [ ]:
# Inspecting what the data types are and if we have any missing values 

In [ ]:
df.info()

In [ ]:
# No missing values apparant, all data seems to be in the correct format. 
# Now I want to check some quick statistics

In [ ]:
df.describe()

In [ ]:
# Now I want to see how the variables are correlated and do some EDA 

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(15,8))
sns.set_style(sns.set_palette('viridis'))
sns.heatmap(df.corr(),annot = True,cmap = 'viridis')

In [ ]:
sns.boxplot(x= 'quality',y = 'alcohol',data = df)

Seems there could be some relationship between the alochol content and the quality of the wine 

In [ ]:
sns.distplot(df.alcohol,bins = 30)

As all values appear to be continuous, it seems fair to leave them all in when trying to predict the quality.
As the values are discrete I think the best way forward would be a decision tree or a Knn, so I will try both and see how the methods get on **

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Start by spliting the data up into test data and training data 

In [ ]:
X = df.drop('quality',axis = 1)
Y = df['quality'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

Im going to set the KNN up in a for loop to produce an elbow curve of the error rate to help me determine what is the best value of K to use 

In [ ]:
error_rate = []
for i in range(1,100):
    KNN = KNeighborsClassifier(n_neighbors=i)
    KNN.fit(X_train,y_train)
    Pred = KNN.predict(X_test)
    error_rate.append(np.mean(Pred != y_test))

In [ ]:
plt.plot(range(1,100),error_rate)

The elbow plot doesn't have a clear minimum, but I guess around 45 would be best for this model. Overall, it appears that the KNN isn't perhaps the best model to use. Lets look at a classification report for a k-value of 45 before moving on to another model.

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=45)
KNN.fit(X_train,y_train)
Pred = KNN.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,Pred))


It seems generally like the KNN classifier isn't doing to great at predicting the quality of wine so I now want to try a random forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF = RandomForestClassifier(n_estimators =250)

In [ ]:
RF.fit(X_train,y_train)

In [ ]:
PredRf = RF.predict(X_test)

In [ ]:
print(classification_report(y_test,PredRf))

It seems like the Random forest does slighly better, however I want to try deep learning to see if that can improve on the methods so far

Code to come

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Dense(11,activation='relu'))
model.add(Dense(11,activation='relu'))
model.add(Dense(11,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=128,epochs=400)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
predictions = model.predict(X_test)

* plt.scatter(y_test,predictions)

In [ ]:
error = y_test.reshape(528,1) - predictions.round(0)

In [ ]:
predictions.round(0)

In [ ]:
y_test.reshape(528,1)

In [ ]:
plt.scatter(y_test,predictions.round(0))